In [ ]:
import wikipedia
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from transformers import pipeline, T5ForConditionalGeneration, T5Tokenizer
import requests
import yfinance as yf
import json
import numpy as np
from collections import defaultdict
from googleapiclient.discovery import build
import os

class ProgressiveNeuralNetwork(nn.Module):
    def __init__(self):
        super(ProgressiveNeuralNetwork, self).__init__()
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')

        self.zero_shot_classifier = pipeline('zero-shot-classification')

        self.qa_model = T5ForConditionalGeneration.from_pretrained('t5-small')
        self.qa_tokenizer = T5Tokenizer.from_pretrained('t5-small')
        self.fc1 = nn.Linear(768, 256) 
        self.fc2 = nn.Linear(256, 128)  
        self.fc3 = nn.Linear(128, 64)   
        self.fc_out = nn.Linear(64, 3)  
        self.cache = defaultdict(str)
        self.feedback_log = []

        # Check if model weights file exists
        self.weights_file = 'model_weights.pth'
        if os.path.exists(self.weights_file):
            # Load the model weights from the file
            self.load_weights()
        else:
            # Initialize and save new model weights if file doesn't exist
            self.save_weights()

        self.google_api_key = ""  # Replace with your Google Search API key
        self.google_search_engine_id = ""  # Replace with your search engine ID (cx)

    def forward(self, query):
        if query in self.cache:
            return self.cache[query]  # Return cached response if available

        # Tokenize and encode the user query
        inputs = self.tokenizer(query, return_tensors='pt', padding=True, truncation=True)
        embeddings = self.bert_model(**inputs).last_hidden_state.mean(dim=1)

        # Forward through the progressive layers
        x = torch.relu(self.fc1(embeddings))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        intent_logits = self.fc_out(x)
        zero_shot_result = self.zero_shot_classifier(query, candidate_labels=["General Knowledge", "Weather", "Stock"])
        print("Zero-Shot Classification Result: ", zero_shot_result)
        highest_score = max(zero_shot_result['scores'])
        predicted_intent = zero_shot_result['labels'][zero_shot_result['scores'].index(highest_score)]

        # Set a confidence threshold to prevent low-confidence predictions
        confidence_threshold = 0.6
        if highest_score < confidence_threshold:
            predicted_intent = "General Knowledge"  # Default to General Knowledge if confidence is low

        print(f"Predicted Intent: {predicted_intent}")
        if predicted_intent == "General Knowledge":
            answer = self.generate_answer_wikipedia(query)
        elif predicted_intent == "Weather":
            answer = self.generate_answer_weather(query)
        elif predicted_intent == "Stock":
            answer = self.generate_answer_stock(query)

        # Cache the response
        self.cache[query] = answer

        # Ask user for feedback dynamically (simulate user input here)
        user_feedback = input(f"Was this answer helpful? (yes/no): ").strip().lower()
        feedback = 1 if user_feedback == "yes" else 0  # Feedback logic (1: Helpful, 0: Not Helpful)

        # Log feedback
        self.log_feedback(query, feedback)

        return answer

    def fetch_from_wikipedia(self, query):
        """Fetch data from Wikipedia, handling disambiguation."""
        try:
            search_results = wikipedia.search(query)
            page_title = None
            for result in search_results:
                if result.lower() == query.lower():
                    page_title = result
                    break
            else:
                page_title = search_results[0] if search_results else None

            if page_title:
                summary = wikipedia.summary(page_title, sentences=5)
                return summary
            else:
                return f"Error fetching data: No relevant Wikipedia page found for '{query}'"

        except Exception as e:
            return f"Error fetching data: {str(e)}"

    def generate_answer_wikipedia(self, query):
        """Fetches information from Wikipedia using the query."""
        return self.fetch_from_wikipedia(query)

    def generate_answer_weather(self, query):
        """Fetches weather info using OpenWeather API or Google Search as fallback."""
        try:
            city = query.split("in")[-1].strip()
            api_key = ''  # Replace with your OpenWeather API key
            url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
            response = requests.get(url)
            data = response.json()

            if response.status_code == 200:
                temperature = data["main"]["temp"] - 273.15
                weather_description = data["weather"][0]["description"]
                return (f"The current weather in {city} is {weather_description} "
                        f"with a temperature of {temperature:.2f}°C.")
            else:
                print("OpenWeather API failed. Using Google Search as fallback.")
                return self.google_search(query)  # Fallback to Google Search
        except Exception as e:
            print(f"Error fetching weather data: {str(e)}. Using Google Search as fallback.")
            return self.google_search(query)  # Fallback to Google Search

    def generate_answer_stock(self, query):
        """Fetches stock info using Alpha Vantage API or Google Search as fallback."""
        try:
            symbol = query.split()[-1].upper()
            api_key = ''  # Replace with your Alpha Vantage API key
            url = (f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}"
                   f"&apikey={api_key}")
            response = requests.get(url)
            data = response.json()

            if "Global Quote" in data:
                global_quote = data["Global Quote"]
                price = global_quote["05. price"]
                change = global_quote["09. change"]
                percent_change = global_quote["10. change percent"]
                return (f"The current price of {symbol} is ${price}. "
                        f"Change: {change} ({percent_change}).")
            else:
                print("Alpha Vantage API failed. Using Google Search as fallback.")
                return self.google_search(query)  # Fallback to Google Search
        except Exception as e:
            print(f"Error fetching stock data: {str(e)}. Using Google Search as fallback.")
            return self.google_search(query)  # Fallback to Google Search

    def google_search(self, query):
        """Performs a Google Search and returns the top result snippet."""
        service = build("customsearch", "v1", developerKey=self.google_api_key)
        result = service.cse().list(q=query, cx=self.google_search_engine_id).execute()

        if "items" in result and result["items"]:
            return result["items"][0]["snippet"]
        else:
            return "No results found on Google Search."

    def log_feedback(self, query, feedback):
        """Logs user feedback to a file for analysis and model adaptation."""
        feedback_data = {
            "query": query,
            "feedback": feedback
        }
        try:
            with open('feedback.json', 'a') as file:
                json.dump(feedback_data, file)
                file.write('\n')
            print("Feedback logged successfully.")
        except Exception as e:
            print(f"Error logging feedback: {str(e)}")

        # Store feedback for future adaptation
        self.feedback_log.append(feedback)

    def update_weights(self, feedback):
        """Adjusts model weights based on feedback."""
        feedback_strength = np.mean(self.feedback_log)  # Calculate mean feedback as a strength metric
        if feedback_strength > 0.8:  # If positive feedback is strong
            print("Adjusting weights based on positive feedback.")
            # Example: increase weights of positive actions
            with torch.no_grad():
                self.fc1.weight += torch.randn_like(self.fc1.weight) * 0.01
                self.fc2.weight += torch.randn_like(self.fc2.weight) * 0.01
                self.fc3.weight += torch.randn_like(self.fc3.weight) * 0.01

        elif feedback_strength < 0.2:  # If negative feedback is strong
            print("Adjusting weights based on negative feedback.")
            # Example: decrease weights of actions that led to mistakes
            with torch.no_grad():
                self.fc1.weight -= torch.randn_like(self.fc1.weight) * 0.01
                self.fc2.weight -= torch.randn_like(self.fc2.weight) * 0.01
                self.fc3.weight -= torch.randn_like(self.fc3.weight) * 0.01

    def adapt_to_feedback(self):
        """Adapts model based on the feedback provided."""
        if len(self.feedback_log) > 10:  # Adapt after accumulating enough feedback
            self.update_weights(self.feedback_log)  # Update model weights based on feedback
            self.feedback_log.clear()  # Clear feedback log after adaptation

    def save_weights(self):
        """Saves the model weights to a file."""
        torch.save({
            'fc1_weight': self.fc1.weight,
            'fc2_weight': self.fc2.weight,
            'fc3_weight': self.fc3.weight
        }, self.weights_file)
        print("Model weights saved successfully.")

    def load_weights(self):
        """Loads the model weights from a file."""
        checkpoint = torch.load(self.weights_file)
        self.fc1.weight.data = checkpoint['fc1_weight']
        self.fc2.weight.data = checkpoint['fc2_weight']
        self.fc3.weight.data = checkpoint['fc3_weight']
        print("Model weights loaded successfully.")

pnn_model = ProgressiveNeuralNetwork()

while True:
    user_query = input("Enter your question (or type 'exit' to quit): ")
    if user_query.lower() == "exit":
        break

    print(f"Query: {user_query}")
    response = pnn_model(user_query)
    print(f"Response: {response}\n")
    pnn_model.adapt_to_feedback()